# **PROJET FINE-TUNING LLM**

## ***Importer les librairies usuelles*** ##

In [ ]:
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install trl
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-edn9496x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-edn9496x


In [ ]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
import random
import torch
import pandas as pd
from numba import cuda
from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer
import re

## ***Importer le datset*** ##

In [ ]:
dataset = load_dataset("manu/french_poetry")

In [ ]:
# Extraire le Datset grâce à la clé "train"
df = dataset['train'].to_pandas()
df.head()

## ***Explorer le Dataset***

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df['text'].iloc[50]

## ***Effectuer le preprocessing sur le dataset*** ##

In [ ]:
# Récupérer la section poème positionnée à l'indice 4
df['text'] = df['text'].str.split(':').str[4]

# Garder la section poème positionnée après le premier "\n"
df['text'] = df['text'].str.split('\n', n=1).str[1]

# Supprimer l'intégralité des NAN (poèmes structure non conforme)
#df['text'] = df['text'].dropna()

In [ ]:
df.head(15)

In [ ]:
df['text'][0]

In [ ]:
df.shape

## ***Suppression des valeurs NaN aprés transformation ne respectant pas la structure globale***

In [ ]:
# Détecter les valeurs NaN
missing_values = df.text.isnull().sum()
print(missing_values)

In [ ]:
# Supprimer les valeurs Nan
df = df.dropna(subset=['text']).reset_index(drop =True)

In [ ]:
df.shape

In [ ]:
# Compter les valeurs NaN après suppression
missing_values = df.text.isnull().sum()
print(missing_values)

In [ ]:
df['title'].iloc[4]


In [ ]:
df['text'].iloc[4]

In [ ]:
# Suppression des colonnes inutiles
df = df.drop(columns = ['poet', 'link', 'id'])

In [ ]:
df.head()

In [ ]:
# Dictionnaire des prompts utilisés pour améliorer la robustesse du dataset
"""prompt_variance = [
    "Écris un poème sur le thème : {title}",
    "Compose un poème intitulé : {title}",
    "Imagine un poème ayant pour sujet : {title}",
    "Crée un poème autour de : {title}",
    "Un poème inspiré de {title}, s'il te plaît",
    "{title}",
    "Rédige un poème poétique à propos de : {title}",
    "Propose un poème sur le sujet suivant : {title}",
    "Donne-moi un poème avec pour thème principal : {title}",
    "Développe un poème basé sur : {title}",
    "Invente un poème portant sur : {title}",
    "Un poème intitulé {title}",
    "Sur le thème {title}, écris un poème captivant",
    "Conçois un poème ayant comme sujet : {title}",
    "Pour {title}, écris un poème unique",
    "À partir du sujet '{title}', rédige un poème",
    "Avec {title}, inspire-toi pour créer un poème",
    "Un poème autour de {title}, si possible",
    "Donne-moi un poème avec pour thème {title}",
    "Fais un poème en explorant le thème {title}",
    "Compose une œuvre poétique sur {title}",
    "Rédige un poème à propos de {title}"
]"""

In [ ]:
"""# Fixer une seed pour la reproductibilité des résultats
random.seed(42)

# Applqiuer les différents prompt aléatoirement sur les titres des poèmes
df['prompt'] = df['title'].apply(lambda title: random.choice(prompt_variance).format(title=title))"""

In [ ]:
# Conserver uniquement les colonnes prompt et text
#df = df[['prompt', 'text']]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.head()

Tokens spéciaux : {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}

## ***Ajouter les token spéciaux au dataset qui ne sont pas présent par déafut dans le tokenizer*** ##

In [ ]:
# token saut de ligne
#df['text'] = df["text"].str.replace("\n", "<|newline|>", regex = False)
#df['text'] = df["text"].str.replace("\n", "<|newline|>", regex = False)

# token padding
# Ajouter des tokens <|pad|> pour chaque ligne
#target_length = 1489

"""df["padded_text"] = df["text"] + df.apply(
    lambda row: " " + " ".join(["<|pad|>"] * (target_length - row["text_length"]))
    if row["text_length"] < target_length else "", axis=1
)"""

In [ ]:
# Conserver uniquement les colonnes prompt et text
#df = df[['prompt', 'text']]

In [ ]:
df.head()

In [ ]:
df.text[5]

In [ ]:
# Convertir le dataset en CSV
df.to_csv('poems_dataset.csv', index=False, encoding='utf-8')

## ***Importer le modèle pour le tester sans fine-tuning*** ##

In [ ]:
#df = pd.read_csv('poems_prompts_dataset.csv')

In [ ]:
"""from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')
generator("EleutherAI has", do_sample=True, min_length=50)"""

In [ ]:
"""prompt = "Génère un poème sur l'amitié"
res = generator(prompt, max_length = 200, top_k = 80, top_p = 0.90, do_sample = True, temperature = 1.1)

print(res[0]['generated_text'])"""

In [ ]:
#generator("Génère un poème sur le thème du courage", do_sample = True, max_length = 200, min_length = 150, temperature = 1, top_k = 80)

In [ ]:
df.shape

In [ ]:
df.text[0]

## ***Begining of NLP*** ##

#***Imports libs here***#

In [ ]:
#from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, default_data_collator
from transformers import DataCollatorForLanguageModeling, default_data_collator
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer

In [ ]:
# Apprentissage masqué (Masked Language Modeling, MLM) pour des modèles comme BERT.
# mlm : Un booléen qui indique si le masquage pour MLM doit être appliqué (par défaut, True).

"""def load_data_collator(tokenizer, mlm = False):
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=mlm, )
  return data_collator"""

In [ ]:
df['combined'] = '[Q]' + df['title'] + ' ' + '[A]' + df['text']

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
# Fractionner en train/test
train_data, test_data = train_test_split(df['combined'], test_size=0.2, random_state=42)

In [ ]:
# Convertir le dataset en datframe hugging face
from datasets import Dataset, DatasetDict

In [ ]:
# Convertir en Dataset Hugging Face
train_dataset = Dataset.from_dict({"text": train_data})
test_dataset = Dataset.from_dict({"text": test_data})
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

In [ ]:
#from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Afficher les token par défaut
# Afficher les tokens spéciaux par défaut
print("Tokens spéciaux par défaut :", tokenizer.special_tokens_map)

In [ ]:
#from transformers import AutoModelForCausalLM
# Ajouter un nouveau token spécial
#tokenizer.add_special_tokens({"unk_token": "<|unknown|>"})
#tokenizer.add_special_tokens({"pad_token": "<|unknown|>"})

# Charger le modèle
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

In [ ]:
print("The max model length is {} for this model".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer

In [ ]:
#tokenizer.all_special_tokens

In [ ]:
#tokenizer.eos_token_id

In [ ]:
#tokenizer.max_model_input_sizes

In [ ]:
"""sentence = 'I am a Data Science and Artificial Inteligence Dev who know to train a model'
input_ids  = tokenizer.encode(sentence, return_tensors = 'pt')"""

In [ ]:
#input_ids

In [ ]:
#tokenizer.decode(input_ids[0][7])

In [ ]:
print("Taille du vocabulaire du tokenizer :", len(tokenizer))
print("Taille du vocabulaire du modèle :", model.config.vocab_size)

In [ ]:
output_dir = './results'
tokenizer.save_pretrained(output_dir)
model.save_pretrained(output_dir)

## ***Ajouter les tokens préalablement créés au tokenizer***

In [ ]:
df.head()

In [ ]:
print(df.columns)

In [ ]:
#df['combined'] = '[Q]' + df['prompt'] + ' ' + '[A]' + df['text']

In [ ]:
#df['combined'] = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [ ]:
train_dataset[0]

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
# Vérifier et définir un token de padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Utiliser eos_token comme pad_token
    # Ou ajouter un nouveau token spécial
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # model.resize_token_embeddings(len(tokenizer))  # Adapter les embeddings

In [ ]:
# Exemple de tokenisation pour vérifier
texts = ["Hello world!", "GPT-2 fine-tuning"]
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

print(encodings)

In [ ]:
# Ajouter le DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Mettez True si vous faites du Masked Language Modeling (par ex., avec BERT)
)

In [ ]:
# Tokenisation
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False)  # Pas de padding ici, géré par le DataCollator

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

emplacement fichier config : accelerate configuration saved at C:\Users\33760/.cache\huggingface\accelerate\default_config.yaml

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets['train']

In [ ]:
"""training_args = TrainingArguments(
    output_dir="./gpt-neo-125M-fine-tuned-poetry",             # Répertoire pour sauvegarder les résultats et checkpoints
    eval_strategy="epoch",       # Évaluer le modèle à la fin de chaque époque
    learning_rate=5e-5,                # Taux d'apprentissage (learning rate)
    per_device_train_batch_size=1,     # Taille du batch par appareil (GPU ou CPU)
    num_train_epochs=1,                # Nombre total d'époques d'entraînement
    save_steps=500,                    # Sauvegarder le modèle toutes les 500 étapes
    save_total_limit=2,                # Conserver uniquement les 2 dernières sauvegardes
    fp16=False,                        # Activer l'entraînement en virgule flottante 16 bits (plus rapide si GPU compatible)
    logging_dir = "./logs",
)"""

In [ ]:
# Importer les modules nécessaires de la bibliothèque transformers
from transformers import TrainingArguments, Trainer

# Configurer les arguments d'entraînement pour le fine-tuning
training_args = TrainingArguments(
    output_dir="./results/trained/",
    overwrite_output_dir=False,
    #debug="underflow_overflow",
    #eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=5,
    #per_device_eval_batch_size=1,
    num_train_epochs=50,
    save_steps=5000,
    #save_total_limit=1,
    fp16=True,  # Utiliser la précision mixte si disponible
    logging_dir='./logs',
    logging_steps=20,
)

# Configurer le Trainer, une classe de Hugging Face pour gérer l'entraînement
trainer = Trainer(
    model=model,                       # Modèle à fine-tuner
    args=training_args,                # Arguments d'entraînement configurés précédemment
    train_dataset=tokenized_datasets["train"],    # Jeu de données tokenisé utilisé pour l'entraînement
    #eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Lancer le processus d'entraînement
trainer.train()

In [ ]:
# Sauvegarder le modèle fine-tuné dans le répertoire : ./gpt-neo-125M-fine-tuned-poetry
trainer.save_model()

In [ ]:
# Sauvegarder le modèle fine-tuné
model.save_pretrained("./gpt2-poetry")
tokenizer.save_pretrained("./gpt2-poetry")

In [ ]:
#generator = pipeline('text-generation', model='./gpt2-poetry', tokenizer='./gpt2-poetry')
#print(generator("Compose un poème sur le bonheur", max_length=200, temperature=0.8))


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(model_path, sequence, max_length):
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
  final_outputs = model.generate(
      ids,
      do_sample=True,
      max_length=max_length,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95,
  )
  print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))


In [ ]:
model2_path = "./gpt2-poetry"
sequence2 = "[Q] un poème sur Nouvel amour "
max_len = 250
generate_text(model2_path, sequence2, max_len)

In [ ]:
#tokenizer.save_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")

In [ ]:
"""def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rouge1'].fmeasure
"""

In [ ]:
"""def generate_poem(model, tokenizer, keyword, max_length=100):
    if not keyword.strip():
        raise ValueError("Le mot-clé ne peut pas être vide. Veuillez entrer un mot valide.")

    input_ids = tokenizer.encode(keyword, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_poem = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_poem"""


In [ ]:
"""def calculate_bleu(reference, candidate):
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
"""

In [ ]:
"""from transformers import GPT2LMHeadModel, GPTNeoForCausalLM, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction"""
#from rouge_score import rouge_scorer

In [ ]:
"""gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

gptneo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
gptneo_tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

tokenizer = AutoTokenizer.from_pretrained("./gpt-neo-125M-fine-tuned-poetry/tokenizer")
model = AutoModelForCausalLM.from_pretrained("./gpt-neo-125M-fine-tuned-poetry")
"""


In [ ]:
"""keyword = input("Entrez un mot-clé pour générer un poème : ").strip()

print(keyword)

if len(keyword) == 0:
    print("Erreur : Le mot-clé ne peut pas être vide.")
    exit()

if len(keyword) > 50:
    print("Erreur : Le mot-clé est trop long. Veuillez entrer un mot ou une courte phrase.")
    exit()

# Générer des poèmes pour chaque modèle
gpt2_poem = generate_poem(gpt2_model, gpt2_tokenizer, keyword)
#gptneo_poem = generate_poem(gptneo_model, gptneo_tokenizer, keyword)
model_poem = generate_poem(model, gptneo_tokenizer, keyword)

# Calculer les scores BLEU et ROUGE
bleu_score = calculate_bleu(gpt2_poem, model_poem)
rouge_score = calculate_rouge(gpt2_poem, model_poem)

# Afficher les poèmes générés et les scores
print("\nPoème généré par GPT-2 :\n", gpt2_poem)
#print("\nPoème généré par GPT-Neo :\n", gptneo_poem)
print("\nPoème généré par model :\n", model_poem)
print("\nScore BLEU entre les poèmes générés : ", bleu_score)
print("Score ROUGE entre les poèmes générés : ", rouge_score)"""
